In [1]:
import pandas as pd
import requests
import config

In [2]:
response_list = []
API_KEY = config.api_key

for movie_id in range(550,556): 
  url = 'https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id, API_KEY)
  r = requests.get(url)
  response_list.append(r.json())

In [4]:
df = pd.DataFrame.from_dict(response_list)
print(df.head())
#the website said that I should be getting 6 rows X 38 columns, but I am only getting 5x25 



   adult                     backdrop_path  \
0  False  /rr7E0NoGKxvbkb89eR1GwfoYjpA.jpg   
1  False  /wMNs0x98yGMjJJ2cGjiR9luINqF.jpg   
2  False  /k4JIHyAXaGHwAwT7y5Skd17f0Wl.jpg   
3  False  /r3xsFBD1VTUusk393bBc7SsDUJe.jpg   
4  False  /1qwXItFKqvKYyW1CwbYhxyUC8Pj.jpg   

                               belongs_to_collection    budget  \
0                                               None  63000000   
1  {'id': 372257, 'name': 'The Poseidon Adventure...   5000000   
2                                               None         0   
3  {'id': 1952, 'name': 'USA: Land of Opportuniti...  10000000   
4                                               None         0   

                                              genres  \
0                      [{'id': 18, 'name': 'Drama'}]   
1  [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...   
2  [{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...   
3  [{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...   
4  [{'id': 18, 'name': 'Drama'}, {'id': 36, 'n

In [5]:

df_columns = ['budget', 'genres', 'id', 'imdb_id', 'original_title', 'release_date', 'revenue', 'runtime']

In [6]:
genres_list = df['genres'].tolist()
flat_list = [item for sublist in genres_list for item in sublist]

result = []
for l in genres_list:
    r = []
    for d in l:
        r.append(d['name'])
    result.append(r)
df = df.assign(genres_all=result)

In [7]:
df_genres = pd.DataFrame.from_records(flat_list).drop_duplicates()

In [8]:
print(df_genres.head())

   id       name
0  18      Drama
1  28     Action
2  12  Adventure
4  53   Thriller
5  35     Comedy


In [10]:
df_columns = ['budget', 'id', 'imdb_id', 'original_title', 'release_date', 'revenue', 'runtime']
df_genre_columns = df_genres['name'].to_list()
df_columns.extend(df_genre_columns)

s = df['genres_all'].explode()
df = df.join(pd.crosstab(s.index, s))


In [12]:
print(df[df_columns])

     budget   id    imdb_id          original_title release_date    revenue  \
0  63000000  550  tt0137523              Fight Club   1999-10-15  100853753   
1   5000000  551  tt0069113  The Poseidon Adventure   1972-12-13   84563118   
2         0  552  tt0237539         Pane e tulipani   2000-03-03    8478434   
3  10000000  553  tt0276919                Dogville   2003-05-19   16680836   
4         0  554  tt0308476                 Кукушка   2002-01-01          0   
5         0  555  tt0442896                 Absolut   2005-04-20          0   

   runtime  Drama  Action  Adventure  Thriller  Comedy  Romance  Crime  \
0      139      1       0          0         0       0        0      0   
1      117      1       1          1         1       0        0      0   
2      114      0       0          0         0       1        1      0   
3      178      1       0          0         1       0        0      1   
4      100      1       0          0         0       1        1      0   
5 

In [13]:
df['release_date'] = pd.to_datetime(df['release_date'])
df['day'] = df['release_date'].dt.day
df['month'] = df['release_date'].dt.month
df['year'] = df['release_date'].dt.year
df['day_of_week'] = df['release_date'].dt.day_name()
df_time_columns = ['id', 'release_date', 'day', 'month', 'year', 'day_of_week']

In [14]:
print(df[df_time_columns])

    id release_date  day  month  year day_of_week
0  550   1999-10-15   15     10  1999      Friday
1  551   1972-12-13   13     12  1972   Wednesday
2  552   2000-03-03    3      3  2000      Friday
3  553   2003-05-19   19      5  2003      Monday
4  554   2002-01-01    1      1  2002     Tuesday
5  555   2005-04-20   20      4  2005   Wednesday
